In [1]:
import pandas as pd
import os
import wiggum as wg
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [2]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the LabeledDataFrame with that DataFrame to create the object

In [3]:
labeled_df_setup = wg.LabeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [4]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [5]:
roles = {'state':['independent','groupby'], 'year':'independent', 'driver_gender':['trend','groupby'], 
         'driver_race':['independent','groupby'],
       'decriminalization':['groupby'], 'medical':['groupby'],
         'recreational':['groupby'], 'no_reforms':['groupby'],
       'search_conducted_false':'ignore', 'search_conducted_true':'ignore',
       'search_conducted_rate':'dependent', 'contraband_found_false':'ignore',
       'contraband_found_true':'ignore', 'contraband_found_rate':'dependent', 'hit_false':'ignore',
       'hit_true':'ignore', 'hit_rate':'dependent', 'num_stops':'dependent'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
var_types = {'driver_gender':'categorical','decriminalization':'categorical', 
             'medical':'categorical', 'recreational':'categorical', 'no_reforms':False,}
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops',
             'contraband_found_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [6]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.set_weighting_vars(weighting)
labeled_df_setup.set_var_types(var_types)
labeled_df_setup.meta_df

/home/smb/anaconda3/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:365: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['role'][k] = v
/home/smb/anaconda3/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:341: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['var_type'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[independent, groupby]",False,NaN
year,int64,ordinal,independent,False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[independent, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,ignore,True,NaN


Now, we've set this up, we can also save these configurations to load them in directly in the future

In [7]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

True

We can see what this does, using a bash magic

In [8]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [9]:
labeled_df = wg.LabeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[independent, splitby]",False,NaN
year,int64,ordinal,[independent],False,NaN
driver_gender,object,categorical,"[[independent, dependent], splitby]",False,NaN
driver_race,object,categorical,"[independent, splitby]",False,NaN
decriminalization,int64,categorical,[splitby],False,NaN
medical,int64,categorical,[splitby],False,NaN
recreational,int64,categorical,[splitby],False,NaN
no_reforms,int64,False,[splitby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [10]:
labeled_df.add_intersectional(['driver_gender','driver_race'])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0,F_White
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0,M_White
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian


We can also pass more variables and lengths of the tuples that we want it to combine.  For example if we provide 3 categorical variables we can do both pairs and triples of the variables.

In [11]:
labeled_df.add_intersectional(['driver_gender','driver_race','state'],[2,3])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race,driver_gender_state,driver_race_state,driver_gender_driver_race_state
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,NaN,NaN,1.0,NaN,NaN,31.0,F_White,F_AZ,White_AZ,F_White_AZ
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black,M_AZ,Black_AZ,M_Black_AZ
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic,M_AZ,Hispanic_AZ,M_Hispanic_AZ
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,0.055556,2.0,2.0,0.5,36.0,M_White,M_AZ,White_AZ,M_White_AZ
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian,F_AZ,Asian_AZ,F_Asian_AZ


In [12]:
labeled_df.add_all_dpgmm(qual_thresh =.2)
labeled_df.df.head()

/home/smb/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race,driver_gender_state,driver_race_state,driver_gender_driver_race_state
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,NaN,NaN,1.0,NaN,NaN,31.0,F_White,F_AZ,White_AZ,F_White_AZ
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black,M_AZ,Black_AZ,M_Black_AZ
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic,M_AZ,Hispanic_AZ,M_Hispanic_AZ
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,0.055556,2.0,2.0,0.5,36.0,M_White,M_AZ,White_AZ,M_White_AZ
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian,F_AZ,Asian_AZ,F_Asian_AZ


In [13]:
labeled_df.add_quantile(['hit_rate','num_stops'])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race,driver_gender_state,driver_race_state,driver_gender_driver_race_state,hit_ratequantiles,num_stopsquantiles
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,1.0,NaN,NaN,31.0,F_White,F_AZ,White_AZ,F_White_AZ,high,low
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,1.0,0.5,3.0,M_Black,M_AZ,Black_AZ,M_Black_AZ,high,low
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,1.0,NaN,NaN,15.0,M_Hispanic,M_AZ,Hispanic_AZ,M_Hispanic_AZ,high,low
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,2.0,0.5,36.0,M_White,M_AZ,White_AZ,M_White_AZ,high,low
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,3.0,NaN,NaN,61.0,F_Asian,F_AZ,Asian_AZ,F_Asian_AZ,high,low


In [14]:
labeled_df.get_vars_per_type('categorical')

['state',
 'driver_gender',
 'driver_race',
 'decriminalization',
 'medical',
 'recreational',
 'driver_gender_driver_race',
 'driver_gender_state',
 'driver_race_state',
 'driver_gender_driver_race_state',
 'hit_ratequantiles',
 'num_stopsquantiles']

In [15]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[independent, splitby]",False,NaN
year,int64,ordinal,[independent],False,NaN
driver_gender,object,categorical,"[[independent, dependent], splitby]",False,NaN
driver_race,object,categorical,"[independent, splitby]",False,NaN
decriminalization,int64,categorical,[splitby],False,NaN
medical,int64,categorical,[splitby],False,NaN
recreational,int64,categorical,[splitby],False,NaN
no_reforms,int64,False,[splitby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [16]:
corrobj = wg.All_Pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

[('year', 'search_conducted_rate'),
 ('year', 'contraband_found_rate'),
 ('year', 'hit_rate'),
 ('year', 'num_stops')]

In [17]:
rankobj = wg.Mean_Rank_Trend()
linreg_obj = wg.All_Linear_Trend()

# Computing Trends on a LabeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [18]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()

,dependent,group_feat,independent,subgroup,subgroup_trend,subgroup_trend_strength,trend_type,agg_trend,agg_trend_strength,comparison_type
0,search_conducted_rate,state,year,AZ,-0.247018,0.247018,pearson_corr,-0.03903,0.03903,aggregate-subgroup
1,search_conducted_rate,driver_gender,year,F,0.030411,0.030411,pearson_corr,-0.03903,0.03903,aggregate-subgroup
2,search_conducted_rate,driver_race,year,Asian,-0.120567,0.120567,pearson_corr,-0.03903,0.03903,aggregate-subgroup
3,search_conducted_rate,decriminalization,year,0,-0.106808,0.106808,pearson_corr,-0.03903,0.03903,aggregate-subgroup
4,search_conducted_rate,medical,year,0,-0.027077,0.027077,pearson_corr,-0.03903,0.03903,aggregate-subgroup


Now we can use a list of objects and apply multiple trends

In [19]:
labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup_trend,subgroup_trend_strength,trend_type
177,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,recreational,year,1,4.84031,0.002812,lin_reg
133,"[WA, NC, IL, AZ, SC, MD, TX, CT, RI, MA, WI, C...",0.219300,aggregate-subgroup,hit_rate,recreational,state,0,"[NC, WA, IL, AZ, SC, MD, TX, CT, RI, MA, WI, C...",0.223900,rank_trend
251,-1560.86,0.043965,aggregate-subgroup,num_stops,driver_race,year,Asian,259.628,0.104919,lin_reg
92,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, WA, RI, I...",0.388100,aggregate-subgroup,search_conducted_rate,driver_gender,state,F,"[CO, NC, VT, CT, MD, MA, WI, SC, TX, RI, WA, I...",0.334800,rank_trend
141,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.320900,aggregate-subgroup,num_stops,driver_race,state,Hispanic,"[VT, RI, WI, MD, CT, SC, IL, MA, NC, CO, WA, A...",0.583900,rank_trend
152,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,state,year,AZ,0.0220027,0.001366,lin_reg
113,"[NC, CO, MD, WA, CT, SC, TX, WI, IL, MA, VT, A...",0.020100,aggregate-subgroup,contraband_found_rate,driver_race,state,White,"[NC, CO, MD, WA, CT, SC, TX, IL, WI, MA, AZ, V...",0.377600,rank_trend
48,"[Asian, Hispanic, Other, Black, White]",0.002300,aggregate-subgroup,hit_rate,state,driver_race,CT,"[Other, Black, Hispanic, White, Asian]",0.255700,rank_trend
81,"[Other, Asian, Black, Hispanic, White]",0.499600,aggregate-subgroup,num_stops,state,driver_race,WI,"[Other, Asian, Hispanic, Black, White]",0.624500,rank_trend
169,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,driver_race,year,Hispanic,5.34096,0.017419,lin_reg


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

We can see what types of trends were computed from `result_df`

In [20]:
pd.unique(labeled_df.result_df['trend_type'])

array(['pearson_corr', 'rank_trend', 'lin_reg'], dtype=object)

In [21]:
pd.unique(labeled_df.result_df['comparison_type'])

array(['aggregate-subgroup'], dtype=object)

We can also add trends that are structured for pairwise comparisons

In [22]:
labeled_df.get_pairwise_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

Index(['independent', 'dependent', 'subgroup_trend', 'subgroup_trend_strength',
       'subgroup', 'group_feat', 'trend_type'],
      dtype='object')
Index(['dependent', 'group_feat', 'independent', 'subgroup', 'subgroup_trend',
       'subgroup_trend_strength', 'trend_type'],
      dtype='object')
546
7186
Index(['dependent', 'group_feat', 'independent', 'subgroup', 'subgroup_trend',
       'subgroup_trend_strength', 'trend_type', 'subgroup2', 'subgroup_trend2',
       'subgroup_trend_strength2'],
      dtype='object')
3056
2312


,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type
27,NaN,NaN,pairwise,search_conducted_rate,state,driver_race,AZ,TX,"[Asian, White, Other, Hispanic, Black]","[Asian, White, Other, Hispanic, Black]",0.035100,0.487300,rank_trend
1236,NaN,NaN,pairwise,contraband_found_rate,state,driver_race,RI,TX,"[Asian, White, Hispanic, Other, Black]","[Asian, Hispanic, White, Other, Black]",0.333400,0.484300,rank_trend
2160,NaN,NaN,pairwise,num_stops,state,driver_race,MA,VT,"[Other, Asian, Hispanic, Black, White]","[Other, Hispanic, Asian, Black, White]",0.514500,0.721400,rank_trend
3776,NaN,NaN,pairwise,search_conducted_rate,state,year,AZ,CO,0.0220027,1.61212,0.001366,0.002736,lin_reg
5970,NaN,NaN,pairwise,num_stops,state,year,NC,WA,2120.79,-5881.23,0.117245,0.102666,lin_reg
1210,NaN,NaN,pairwise,contraband_found_rate,state,driver_race,NC,TX,"[Asian, White, Other, Black, Hispanic]","[Asian, Hispanic, White, Other, Black]",0.235400,0.484300,rank_trend
6270,NaN,NaN,pairwise,num_stops,state,year,MD,WI,-1571.81,-1188.85,0.136236,0.081042,lin_reg
2588,NaN,NaN,pairwise,num_stops,state,driver_race,RI,TX,"[Other, Asian, Hispanic, Black, White]","[Other, Asian, Black, Hispanic, White]",0.799000,0.923800,rank_trend
1066,NaN,NaN,pairwise,contraband_found_rate,state,driver_race,CO,SC,"[Asian, White, Other, Hispanic, Black]","[Other, White, Black, Hispanic]",0.199200,0.142200,rank_trend
1900,NaN,NaN,pairwise,hit_rate,state,driver_race,RI,VT,"[Other, Hispanic, Asian, Black, White]","[Hispanic, Other, Asian, Black, White]",0.099600,0.144700,rank_trend


In [23]:
pd.unique(labeled_df.result_df['comparison_type'])

array(['aggregate-subgroup', 'pairwise'], dtype=object)

The object also stores the trend objects that have been applied, they can be used for mapping to get the distance functions that are appropriate for each trend

In [24]:
labeled_df.trend_list

In [25]:
labeled_df.result_df.head()

,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type
0,-0.0390297,0.03903,aggregate-subgroup,search_conducted_rate,state,year,AZ,NaN,-0.247018,NaN,0.247018,NaN,pearson_corr
1,-0.0390297,0.03903,aggregate-subgroup,search_conducted_rate,driver_gender,year,F,NaN,0.0304108,NaN,0.030411,NaN,pearson_corr
2,-0.0390297,0.03903,aggregate-subgroup,search_conducted_rate,driver_race,year,Asian,NaN,-0.120567,NaN,0.120567,NaN,pearson_corr
3,-0.0390297,0.03903,aggregate-subgroup,search_conducted_rate,decriminalization,year,0,NaN,-0.106808,NaN,0.106808,NaN,pearson_corr
4,-0.0390297,0.03903,aggregate-subgroup,search_conducted_rate,medical,year,0,NaN,-0.0270766,NaN,0.027077,NaN,pearson_corr


In [26]:
# a = ['a','b','c','d']
# b = ['a','b','d','e']
a = ['F', 'M']
b = ['F', 'M']
for i, d in enumerate(set(a+b)):
    print (i,':',d)

0 : M
1 : F


In [27]:
# labeled_df.result_df['distance'] = labeled_df.result_df.apply(dist_helper,axis=1)
labeled_df.add_distance(row_wise=True) #('subgroup_trend','subgroup_trend2')

labeled_df.result_df.sample(10)

,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance
3315,NaN,NaN,pairwise,hit_rate,decriminalization,state,0,1,"[WA, IL, AZ, MD, SC, RI, MA, WI, VT]","[NC, TX, CT, MD, RI, MA, CO, VT]",0.444900,0.082400,rank_trend,0.542300
576,NaN,NaN,pairwise,search_conducted_rate,state,driver_race,SC,WI,"[Other, White, Black, Hispanic]","[Asian, White, Hispanic, Other, Black]",0.414300,0.244700,rank_trend,0.447300
67,NaN,NaN,pairwise,search_conducted_rate,state,driver_race,CO,VT,"[Asian, White, Other, Hispanic, Black]","[Asian, White, Hispanic, Black, Other]",0.328100,0.022600,rank_trend,0.600000
3174,NaN,NaN,pairwise,num_stops,driver_race,state,Hispanic,White,"[VT, RI, WI, MD, CT, SC, IL, MA, NC, CO, WA, A...","[RI, VT, CT, WI, MD, AZ, MA, CO, IL, NC, SC, W...",0.583900,0.633300,rank_trend,0.435900
2757,NaN,NaN,pairwise,contraband_found_rate,driver_gender,state,F,M,"[NC, CO, MD, CT, WA, SC, TX, MA, WI, VT, IL, R...","[NC, CO, WA, MD, CT, TX, SC, IL, WI, MA, AZ, V...",0.040000,0.201300,rank_trend,0.371800
4213,NaN,NaN,pairwise,search_conducted_rate,state,year,MA,TX,4.37932,3.76278,0.015387,0.014346,lin_reg,0.022447
142,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209,aggregate-subgroup,num_stops,driver_race,state,Other,NaN,"[VT, RI, IL, CT, WI, CO, TX, MD, MA, SC, WA, N...",NaN,0.575200,NaN,rank_trend,0.294900
1796,NaN,NaN,pairwise,hit_rate,state,driver_race,IL,VT,"[Hispanic, Asian, Other, Black, White]","[Hispanic, Other, Asian, Black, White]",0.326600,0.144700,rank_trend,0.100000
3218,NaN,NaN,pairwise,num_stops,driver_race,state,Black,Other,"[VT, RI, WI, CO, CT, AZ, MA, WA, MD, IL, NC, T...","[VT, RI, IL, CT, WI, CO, TX, MD, MA, SC, WA, N...",0.768800,0.575200,rank_trend,0.525600
235,NaN,NaN,pairwise,search_conducted_rate,state,driver_race,SC,TX,"[Other, White, Black, Hispanic]","[Asian, White, Other, Hispanic, Black]",0.414300,0.487300,rank_trend,0.131100


Each trend object has a trend_precompute dictionary as a property that stores the intermediate values (tables of the weighted rates for ranks and correlation matrices for pearson correlation, TODO: what do we need for linreg). These can be used in vizualization.

In [28]:
labeled_df.trend_list[0].trend_precompute

{'pearson_corr_agg_trend':                        search_conducted_rate  contraband_found_rate  hit_rate  \
 search_conducted_rate               1.000000               0.807915 -0.094273   
 contraband_found_rate               0.807915               1.000000  0.275397   
 hit_rate                           -0.094273               0.275397  1.000000   
 num_stops                          -0.065594              -0.073456 -0.042102   
 year                               -0.039030               0.041129  0.316196   
 
                        num_stops      year  
 search_conducted_rate  -0.065594 -0.039030  
 contraband_found_rate  -0.073456  0.041129  
 hit_rate               -0.042102  0.316196  
 num_stops               1.000000 -0.043965  
 year                   -0.043965  1.000000  ,
 'pearson_corr_subgroup_trend':                                   search_conducted_rate  \
 no_reforms                                                
 0          search_conducted_rate               1.00

# Saving with trends

In [29]:
labeled_df.save_all('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI_all')

True

In [30]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI_all
ls

df.csv
meta.csv
result_df.csv
trends.json


In [31]:
labeled_df = wg.LabeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI_all')

/home/smb/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/home/smb/anaconda3/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:291: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['trend_type'] = tt


In [32]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[independent, splitby]",False,NaN
year,int64,ordinal,[independent],False,NaN
driver_gender,object,categorical,"[independent, dependent, splitby]",False,NaN
driver_race,object,categorical,"[independent, splitby]",False,NaN
decriminalization,int64,categorical,[splitby],False,NaN
medical,int64,categorical,[splitby],False,NaN
recreational,int64,categorical,[splitby],False,NaN
no_reforms,int64,False,[splitby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [33]:
labeled_df.trend_list[0].trend_precompute

{'pearson_corr_agg_trend':    search_conducted_rate  contraband_found_rate  hit_rate  num_stops      year
 0               1.000000               0.807915 -0.094273  -0.065594 -0.039030
 1               0.807915               1.000000  0.275397  -0.073456  0.041129
 2              -0.094273               0.275397  1.000000  -0.042102  0.316196
 3              -0.065594              -0.073456 -0.042102   1.000000 -0.043965
 4              -0.039030               0.041129  0.316196  -0.043965  1.000000,
 'pearson_corr_subgroup_trend':    search_conducted_rate  contraband_found_rate  hit_rate  num_stops      year
 0               1.000000               0.713713 -0.099515  -0.050946  0.080876
 1               0.713713               1.000000  0.392352  -0.079631  0.118325
 2              -0.099515               0.392352  1.000000  -0.055962  0.315970
 3              -0.050946              -0.079631 -0.055962   1.000000 -0.069907
 4               0.080876               0.118325  0.315970  -0

# Filtering

In [34]:
help(labeled_df.get_trend_rows)

Help on method get_trend_rows in module wiggum.ranking_processing:

get_trend_rows(independent=None, dependent=None, group_feat=None, subgroup=None, subgroup2=None, trend_type=None, comparison_type=None, inplace=False) method of wiggum.labeled_dataframe.LabeledDataFrame instance
    return a row of result_df based on the specified values. returned rows
    meet provided criteria for all columns (and operator) and any one of the listed
    values for each column (or operator)
    
    Parameters
    -----------
    indep : str, list, or  {None}
        trend variable name or None to include all
    dependent : str, list, or  {None}
        trend variable name or None to include all
    group_feat : str, list, or  {None}
        groupoby variable name or None to include all
    subgroup : str, list, or  {None}
        value of groupby_feat or  None to include all
    subgroup2 : str, list or {None}
        value of groupby_feat or  None to include all
    trend_type: str, list or {None}


So, we can use that function to filter and look at subsets of the trends based on the features, groupby, or subgroups

In [35]:
labeled_df.get_trend_rows(feat1='year',subgroup=['Black','Hispanic'])

TypeError: get_trend_rows() got an unexpected keyword argument 'feat1'

In [ ]:
labeled_df.get_trend_rows(group_feat = 'driver_race',trend_type ='lin_reg' )

In [ ]:
labeled_df.result_df.columns

We can also filter based on SP detections with `

In [ ]:
labeled_df.get_SP_rows(thresh=.2)

## Detection

Detection via `get_SP_rows` happens in two steps:
1. label the rows
2. filter by that column to return

Labeling the rows can happen in a number of ways too, the detection accepts a number of forms of input, custom detections can be built in many ways

In [ ]:
help(labeled_df.label_SP_rows)

when filter_thresh is a dictionary, the filtering happens by taking the intersection of each row by the treshold prvided.  Some defaults are also built in accessible by string.

In [ ]:
wg.trend_quality_sp

Which can be applied with:

In [ ]:
labeled_df.get_SP_rows('default_qual_sp')

In [ ]:
wg.DEFAULT_SP_DEF

Which can be applied with:

In [ ]:
labeled_df.get_SP_rows('SP')

We can also define our own detection filters, using any available column

In [ ]:
lin_only_qual = {'name':'lin_only_qual_sp','distance':.2, 'agg_trend_strength':.05,
                'subgroup_trend_strength':.15,'trend_type':'lin_reg'}
labeled_df.get_SP_rows(lin_only_qual,replace=True) 

# Ranking

In [ ]:
labeled_df.rank_occurences_by_view(ascending=False).head(20)

In [ ]:
labeled_df.add_view_score('SP_thresh0.2',agg_type='sum',colored=False).head(10)

In [ ]:
labeled_df.rank_occurences_by_view('sum_view_SP_thresh0.2','SP_thresh0.2').head()